In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pii-detection-removal-from-educational-data/sample_submission.csv
/kaggle/input/pii-detection-removal-from-educational-data/train.json
/kaggle/input/pii-detection-removal-from-educational-data/test.json


#  1.Configuração das bibliotecas

In [2]:
!pip install faker transformers accelerate -q

import pandas as pd
import random
import torch
from faker import Faker
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from torch.utils.data import Dataset
import matplotlib.pyplot as plt

# Configura Faker para dados brasileiros reais
fake = Faker('pt_BR')
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Ambiente pronto no: {device}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.5 MB/s eta 0:00:00:00:0100:01
ERROR: pip's depend

2025-11-28 11:26:48.979740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764329209.207293      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764329209.268107      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

✅ Ambiente pronto no: cuda


# 2.Geração de Dados

In [3]:
print("🏭 Iniciando Geração Massiva de Dados Hierárquicos...")

# --- 1. AS REGRAS DA METODOLOGIA---
def aplicar_politica(tipo, valor):
    """
    Regras de Anonimização conforme Seção III:
    1. Crítico (CPF, Senha, Fone, Cartão) -> [REMOVIDO]
    2. Alto (Nome, Email)                 -> [ETIQUETA]
    3. Baixo (Cidade, Cargo, Empresa)     -> MANTER ORIGINAL
    """
    # Nível 1: Crítico (Supressão)
    if tipo in ['cpf', 'senha', 'telefone', 'cartao', 'rg']:
        return "[REMOVIDO]"
    
    # Nível 2: Alto (Generalização)
    if tipo == 'nome': return "[NOME]"
    if tipo == 'email': return "[EMAIL]"
    
    # Nível 3: Baixo (Manutenção do Contexto)

    return valor 

# --- 2.TEMPLATES ---
templates_banco = [
    "O cliente {nome} (CPF {cpf}) realizou um saque em {cidade}.",
    "Bloqueio preventivo do cartão {cartao} pertencente a {nome}.",
    "A gerente {gerente} autorizou o aumento de limite para {nome}.",
    "Transação suspeita: {nome} tentou usar o cartão {cartao} em {cidade}.",
    "Favor atualizar o cadastro de {nome}. Novo telefone: {telefone}."
]

templates_hospital = [
    "Paciente {nome}, residente em {cidade}, diagnosticado com gripe.",
    "O médico {medico} solicitou exames para {nome} (RG {rg}).",
    "Encaminhar resultado para o email {email} urgente.",
    "Paciente {nome} internado na UTI do hospital de {cidade}.",
    "Ligar para o responsável de {nome} no número {telefone}."
]

templates_rh = [
    "O funcionário {nome} foi promovido a {cargo} na filial de {cidade}.",
    "Login de rede criado para {nome}. Senha temporária: {senha}.",
    "Candidato {nome} aprovado para a vaga de {cargo}.",
    "Enviar holerite para o email corporativo: {email}.",
    "O colaborador {nome} esqueceu o crachá na portaria."
]

templates_juridico = [
    "Processo movido por {nome} contra a empresa de {cidade}.",
    "O réu {nome}, portador do CPF {cpf}, nega as acusações.",
    "A advogada {advogada} solicitou a revisão da pena de {nome}.",
    "Intimação enviada para o endereço de {nome} em {cidade}.",
    "Audiência de conciliação entre {nome} e a parte contrária."
]

# --- 3. GERAÇÃO---
data_pairs = []

num_exemplos = 20000 

print(f"⏳ Gerando {num_exemplos} frases únicas...")

for _ in range(num_exemplos):
    dados = {
        'nome': fake.name(),
        'cpf': fake.cpf(),
        'rg': fake.rg(),
        'cidade': fake.city(),
        'telefone': fake.phone_number(),
        'email': fake.email(),
        'senha': fake.password(),
        'cartao': fake.credit_card_number(),
        'cargo': fake.job(),
        'medico': "Dr. " + fake.first_name(),
        'gerente': fake.first_name(),
        'advogada': "Dra. " + fake.first_name()
    }
    
    # Escolhe um contexto aleatório
    contexto = random.choice(['banco', 'hospital', 'rh', 'juridico'])
    
    if contexto == 'banco': template = random.choice(templates_banco)
    elif contexto == 'hospital': template = random.choice(templates_hospital)
    elif contexto == 'rh': template = random.choice(templates_rh)
    else: template = random.choice(templates_juridico)
    
    # Cria o Input (Preenche com dados reais)
    input_text = template.format(**dados)
    
    # Cria o Target (Preenche aplicando a política de privacidade)
    dados_tratados = {k: aplicar_politica(k, v) for k, v in dados.items()}
    target_text = template.format(**dados_tratados)
    
    data_pairs.append([f"anonimizar: {input_text}", target_text])

# Cria DataFrame
df = pd.DataFrame(data_pairs, columns=['input_text', 'target_text'])
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

print(f"✅ SUCESSO! Dataset com {len(df)} frases gerado.")
print("\n--- AMOSTRA DOS DADOS (VERIFIQUE A HIERARQUIA) ---")
pd.set_option('display.max_colwidth', None)

print(df.sample(5))

🏭 Iniciando Geração Massiva de Dados Hierárquicos...
⏳ Gerando 20000 frases únicas...
✅ SUCESSO! Dataset com 20000 frases gerado.

--- AMOSTRA DOS DADOS (VERIFIQUE A HIERARQUIA) ---
                                                                                                  input_text  \
17958                 anonimizar: Bloqueio preventivo do cartão 4209971853213638 pertencente a Samuel Lopes.   
4667                  anonimizar: O réu José Miguel Rios, portador do CPF 754.283.106-26, nega as acusações.   
15851                      anonimizar: A advogada Dra. Emilly solicitou a revisão da pena de Júlia Carvalho.   
17903  anonimizar: Transação suspeita: Sr. Luigi da Paz tentou usar o cartão 501898139743 em Barbosa do Sul.   
10695                   anonimizar: Login de rede criado para Davi Luiz Viana. Senha temporária: y7LBc8Oj*5.   

                                                                         target_text  
17958                 Bloqueio preventivo do cartão [REMOV

# 3.Tokenização (Preparando dados)

In [4]:
tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)

class HierarquiaDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=128): 
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.data)
    def __getitem__(self, index):
        row = self.data.iloc[index]
        inputs = self.tokenizer(row['input_text'], max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt")
        targets = self.tokenizer(row['target_text'], max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt")
        return {'input_ids': inputs['input_ids'].flatten(), 'attention_mask': inputs['attention_mask'].flatten(), 'labels': targets['input_ids'].flatten()}

train_dataset = HierarquiaDataset(train_df, tokenizer)
val_dataset = HierarquiaDataset(val_df, tokenizer)
print("✅ Tokenização concluída.")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

✅ Tokenização concluída.


# 4.Treinamento

In [5]:
print("🚀 Iniciando Treinamento (20k frases, 5 épocas)...")
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

training_args = Seq2SeqTrainingArguments(
    output_dir="./resultado_hierarquia",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=3e-4,
    weight_decay=0.01,
    logging_steps=200,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    tokenizer=tokenizer,
)

trainer.train()

print("\n💾 Salvando modelo final...")
model.save_pretrained("./modelo_final_1")
tokenizer.save_pretrained("./modelo_final_1")
print("✅ Modelo Salvo!")

🚀 Iniciando Treinamento (20k frases, 5 épocas)...


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/tmp/ipykernel_47/1675320129.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input ten

Epoch,Training Loss,Validation Loss
1,0.158000,0.000000
2,0.000100,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
5,0.000000,0.000000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



💾 Salvando modelo final...
✅ Modelo Salvo!


# 5.Modo Interativo

In [6]:
import torch
import pandas as pd

print("🛡️ Carregando IA Treinada para Benchmark Completo...")
model_final = T5ForConditionalGeneration.from_pretrained("./modelo_final_1").to(device)
tokenizer_final = T5Tokenizer.from_pretrained("./modelo_final_1", legacy=False)

def anonimizar(texto):
    inputs = tokenizer_final(f"anonimizar: {texto}", return_tensors="pt").input_ids.to(device)
    outputs = model_final.generate(inputs, max_length=128, num_beams=2, early_stopping=True)
    return tokenizer_final.decode(outputs[0], skip_special_tokens=True)

print("\n" + "="*80)
print("🎓 BENCHMARK DE ESTRESSE: DO FÁCIL AO IMPOSSÍVEL")
print("="*80)

testes = [
    # --- NÍVEL 1: FÁCIL (O que ela viu no treino) ---
    ("FÁCIL", "O cliente João Silva tem o CPF 111.222.333-44."),
    ("FÁCIL", "A funcionária Maria Oliveira mora em Curitiba."),
    ("FÁCIL", "Login: admin. Senha: password123."),
    ("FÁCIL", "Favor enviar email para suporte@empresa.com.br."),
    
    # --- NÍVEL 2: MÉDIO (Variações de Formato) ---
    ("MÉDIO", "Meu CPF é 04925133330 (sem pontuação)."),
    ("MÉDIO", "Ligar para (19) 99888-7777 urgente."),
    ("MÉDIO", "O Dr. Carlos atende no Hospital Sírio-Libanês."),
    ("MÉDIO", "Acesso liberado para roberto.santos@gmail.com."),
    
    # --- NÍVEL 3: DIFÍCIL (Ambiguidade e Contexto Curto) ---
    ("DIFÍCIL", "O Pedro foi promovido."), 
    ("DIFÍCIL", "A Rosa comprou flores."), 
    ("DIFÍCIL", "Falar com o Leão."),
    ("DIFÍCIL", "Eu sou Arlyson."),
    
    ("HARDCORE", "Lebron James joga no Los Angeles Lakers."), 
    ("HARDCORE", "O juiz Sergio Moro e o ex-presidente Lula."), 
    ("HARDCORE", "A empresa Ford demitiu o funcionário Ford."), 
    ("HARDCORE", "Meu nome é Null e meu CPF é zero."),     
]

resultados = []

for nivel, frase in testes:
    saida = anonimizar(frase)
    print(f"[{nivel}]")
    print(f"Entrada: {frase}")
    print(f"Saída:   {saida}")
    print("-" * 80)
    
    resultados.append({'Nível': nivel, 'Entrada': frase, 'Saída': saida})

df_res = pd.DataFrame(resultados)

🛡️ Carregando IA Treinada para Benchmark Completo...

🎓 BENCHMARK DE ESTRESSE: DO FÁCIL AO IMPOSSÍVEL
[FÁCIL]
Entrada: O cliente João Silva tem o CPF 111.222.333-44.
Saída:   O cliente [NOME] tem o CPF [REMOVIDO].
--------------------------------------------------------------------------------
[FÁCIL]
Entrada: A funcionária Maria Oliveira mora em Curitiba.
Saída:   A funcionária [NOME] mora em Curitiba.
--------------------------------------------------------------------------------
[FÁCIL]
Entrada: Login: admin. Senha: password123.
Saída:   Login: [NOME]. Senha: password123.
--------------------------------------------------------------------------------
[FÁCIL]
Entrada: Favor enviar email para suporte@empresa.com.br.
Saída:   Favor enviar email para suporte@empresa.com.br.
--------------------------------------------------------------------------------
[MÉDIO]
Entrada: Meu CPF é 04925133330 (sem pontuação).
Saída:   Meu CPF é [REMOVIDO] (sem pontuaço).
-------------------------------

In [ ]:
import torch

# Garante que o modelo está carregado na GPU
model_final = T5ForConditionalGeneration.from_pretrained("./modelo_final_1").to(device)
tokenizer_final = T5Tokenizer.from_pretrained("./modelo_final_1", legacy=False)

def anonimizar_frase(texto):
    # 1. Prepara o texto
    entrada = f"anonimizar: {texto}"
    inputs = tokenizer_final(entrada, return_tensors="pt").input_ids.to(device)
    
    # 2. Gera a resposta
    outputs = model_final.generate(
        inputs, 
        max_length=128, 
        num_beams=2,       
        early_stopping=True
    )
    
    # 3. Decodifica
    return tokenizer_final.decode(outputs[0], skip_special_tokens=True)

print("\n" + "="*60)
print("💻 MODO INTERATIVO ATIVADO")
print("Digite uma frase e aperte ENTER para ver o resultado.")
print("Digite 'SAIR' para encerrar.")
print("="*60)

while True:
    try:
        frase_usuario = input("\n📝 Digite a frase: ")
        
        # Condição de saída
        if frase_usuario.lower() in ['sair', 'exit', 'pare']:
            print("\n🛑 Encerrando sistema...")
            break
        
        if len(frase_usuario.strip()) == 0:
            continue

        resultado = anonimizar_frase(frase_usuario)
        
        print("-" * 60)
        print(f"🔒 RESULTADO ANONIMIZADO:")
        print(f"   {resultado}")
        print("-" * 60)
        
    except KeyboardInterrupt:
        print("\n🛑 Sistema interrompido.")
        break
    except Exception as e:
        print(f"❌ Erro: {e}")


💻 MODO INTERATIVO ATIVADO
Digite uma frase e aperte ENTER para ver o resultado.
Digite 'SAIR' para encerrar.



📝 Digite a frase:  Solicito com urgência a atualização cadastral do novo Gerente de Vendas, Sr. Roberto Almeida, portador do CPF 123.456.789-00, que irá atuar na filial de Curitiba, sendo que suas credenciais de acesso foram redefinidas para o usuário 'roberto.a' com a senha provisória Mudar@2024 e o telefone corporativo é (11) 98765-4321.


------------------------------------------------------------
🔒 RESULTADO ANONIMIZADO:
   Solicito com urgência a atualizaço cadastral do novo [NOME], portador do CPF [REMOVIDO], sendo que suas credenciais de acesso foram redefinidas para o usuário [REMOVIDO] e o telefone corporativo é [REMOVIDO].
------------------------------------------------------------



📝 Digite a frase:  O cliente Marcos Souza entrou em contato via chat informando que perdeu o cartão de crédito final 4321 e que criminosos tentaram acessar sua conta com a senha 123456, por favor enviar o token de recuperação para o email marcos.souza@gmail.com ou para o celular (21) 99999-8888 imediatamente.


------------------------------------------------------------
🔒 RESULTADO ANONIMIZADO:
   O cliente [NOME] entrou em contato via chat informando que perdeu o carto de crédito final [REMOVIDO] e que criminosos tentaram acessar sua conta com a senha [REMOVIDO], por favor enviar o token de recuperaço para o email [REMOVIDO] imediatamente.
------------------------------------------------------------



📝 Digite a frase:  "Prezados, informo que o Engenheiro Sênior Carlos Eduardo, residente em Belo Horizonte, solicitou a redefinição urgente da senha de rede 'Mudar#2024' pois seu acesso foi bloqueado após três tentativas falhas usando o CPF 055.444.333-22 como login, sendo necessário enviar o token de validação para o email corporativo c.eduardo@construtora.com.br ou entrar em contato pelo telefone (31) 99876-5432 imediatamente."


------------------------------------------------------------
🔒 RESULTADO ANONIMIZADO:
   Prezados, informo que o Engenheiro [NOME], residente em Belo Horizonte, solicitou a redefiniço urgente da senha de rede [REMOVIDO], bloqueado após três tentativas falhas usando o CPF [REMOVIDO], sendo necessário enviar o token de validaço para
------------------------------------------------------------
